In [1]:
# coding: UTF-8
# use model "chinese_roberta_wwm_ext_pytorch"
import torch
import time 
import torch.nn as nn
import torch.nn.functional as F 
from pytorch_pretrained_bert import BertModel, BertTokenizer, BertConfig, BertAdam
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
from torch.utils.data import *
#CUDA_DEVICE_ORDER="PCI_BUS_ID"  CUDA_VISIBLE_DEVICES=1 python function.py

In [2]:
path = "data/"
bert_path = "chinese_roberta_wwm_ext_pytorch/"
tokenizer = BertTokenizer(vocab_file=bert_path + "vocab.txt")  # 初始化分词器

input_ids = []     # input char ids
input_types = []   # segment ids
input_masks = []   # attention mask
label = []         # 标签
pad_size = 38   # 也称为 max_len (前期统计分析，文本长度最大值为38，取32即可覆盖99%)
 
with open(path + "train.txt", encoding='utf-8') as f:
    for i, l in tqdm(enumerate(f)): 
        x1, y = l.strip().split('\t')
        x1 = tokenizer.tokenize(x1)
        tokens = ["[CLS]"] + x1 + ["[SEP]"]
        
        # 得到input_id, seg_id, att_mask
        ids = tokenizer.convert_tokens_to_ids(tokens)
        types = [0] *(len(ids))
        masks = [1] * len(ids)
        # 短则补齐，长则切断
        if len(ids) < pad_size:
            types = types + [1] * (pad_size - len(ids))  # mask部分 segment置为1
            masks = masks + [0] * (pad_size - len(ids))
            ids = ids + [0] * (pad_size - len(ids))
        else:
            types = types[:pad_size]
            masks = masks[:pad_size]
            ids = ids[:pad_size]
        input_ids.append(ids)
        input_types.append(types)
        input_masks.append(masks)
#         print(len(ids), len(masks), len(types)) 
        assert len(ids) == len(masks) == len(types) == pad_size
        label.append([int(y)])
#print(label)

124691it [00:15, 8122.99it/s]


In [3]:
# 随机打乱索引
random_order = list(range(len(input_ids)))
np.random.seed(2020)   # 固定种子
np.random.shuffle(random_order)
#print(random_order[:21])

# 4:1 划分训练集和测试集
input_ids_train = np.array([input_ids[i] for i in random_order[:int(len(input_ids)*0.8)]])
input_types_train = np.array([input_types[i] for i in random_order[:int(len(input_ids)*0.8)]])
input_masks_train = np.array([input_masks[i] for i in random_order[:int(len(input_ids)*0.8)]])
y_train = np.array([label[i] for i in random_order[:int(len(input_ids) * 0.8)]])
print(input_ids_train.shape, input_types_train.shape, input_masks_train.shape, y_train.shape)

input_ids_test = np.array([input_ids[i] for i in random_order[int(len(input_ids)*0.8):]])
input_types_test = np.array([input_types[i] for i in random_order[int(len(input_ids)*0.8):]])
input_masks_test = np.array([input_masks[i] for i in random_order[int(len(input_ids)*0.8):]])
y_test = np.array([label[i] for i in random_order[int(len(input_ids) * 0.8):]])
print(input_ids_test.shape, input_types_test.shape, input_masks_test.shape, y_test.shape)

BATCH_SIZE = 24
train_data = TensorDataset(torch.LongTensor(input_ids_train), 
                           torch.LongTensor(input_types_train), 
                           torch.LongTensor(input_masks_train), 
                           torch.LongTensor(y_train))
train_sampler = RandomSampler(train_data)  
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(torch.LongTensor(input_ids_test), 
                          torch.LongTensor(input_types_test), 
                         torch.LongTensor(input_masks_test),
                          torch.LongTensor(y_test))
test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

(99752, 38) (99752, 38) (99752, 38) (99752, 1)
(24939, 38) (24939, 38) (24939, 38) (24939, 1)


In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(bert_path)  # /bert_pretrain/
        for param in self.bert.parameters():
            param.requires_grad = True  # 每个参数都要 求梯度 feiyu 改成false
        self.fc = nn.Linear(768, 21)   # 768 -> 2

    def forward(self, x):
        context = x[0]  # 输入的句子   (ids, seq_len, mask)
        types = x[1]
        mask = x[2]  # 对padding部分进行mask，和句子相同size，padding部分用0表示，如：[1, 1, 1, 1, 0, 0]
        _, pooled = self.bert(context, token_type_ids=types, 
                              attention_mask=mask, 
                              output_all_encoded_layers=False) # 控制是否输出所有encoder层的结果
        out = self.fc(pooled)   # 得到10分类
        return out

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DEVICE = 'cpu'
print(DEVICE)
model = Model().to(DEVICE)

print(model) 

cuda
Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
        

In [5]:
param_optimizer = list(model.named_parameters())  # 模型参数名字列表
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
# optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
NUM_EPOCHS = 1
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=0.05,
                     t_total=len(train_loader) * NUM_EPOCHS
                    )

def select_predict(test_y,pred_y,device):
    select_y_pred=torch.Tensor(pred_y.shape[0],3).to(device)
    for i in range(pred_y.shape[0]):
        pos = test_y[i]//3 ## 找到对应的位置
        select_y_pred[i] = pred_y[i][pos:pos+3]
        #select_y_pred = torch.cat((select_y_pred,y_select),0)
    return select_y_pred

In [6]:
def train(model, device, train_loader, optimizer, epoch):   # 训练模型
    model.train()
    best_acc = 0.0 
    for batch_idx, (x1,x2,x3, y) in enumerate(train_loader):
        start_time = time.time()
        x1,x2,x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        y_pred = model([x1, x2, x3])  # 得到预测结果
        model.zero_grad()             # 梯度清零
        loss = F.cross_entropy(y_pred, y.squeeze())  # 得到loss
        loss.backward()
        optimizer.step()
        if(batch_idx + 1) % 100 == 0:    # 打印loss
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'.format(epoch, (batch_idx+1) * len(x1), 
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader), 
                                                                           loss.item()))  # 记得为loss.item()


In [7]:
def select_predict(test_y,pred_y):
    select_y_pred=torch.Tensor(pred_y.shape[0],3)
    for i in range(pred_y.shape[0]):
        pos = int(test_y[i]//3) ## 找到对应的位置
        select_y_pred[i] = pred_y[i][pos:pos+3]
        #select_y_pred = torch.cat((select_y_pred,y_select),0)
    return select_y_pred
 
def Normalization(test_y):
    y_norm = torch.Tensor(test_y.shape[0],1)
    for i in range(test_y.shape[0]):
        y_norm[i] = int(test_y[i] % 3) # 转换为 int
    return y_norm

In [8]:
def test(model, device, test_loader):    # 测试模型, 得到测试集评估结果
    model.eval()
    test_loss = 0.0 
    acc = 0 

    for batch_idx, (x1,x2,x3, y) in enumerate(test_loader):
        x1,x2,x3, y_origin = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        y = Normalization(y_origin).to(device)
        y = y.type_as(y_origin)
        with torch.no_grad():#
            y_pred = model([x1,x2,x3])      
        y_ = select_predict(y,y_pred).to(device)
        y_= y_.type_as(y_pred)
        test_loss += F.cross_entropy(y_, y.squeeze())
        pred = y_.max(-1, keepdim=True)[1]   # .max(): 2输出，分别为最大值和最大值的index
        acc += pred.eq(y.view_as(pred)).sum().item()    # 记得加item()

      

    test_loss /= len(test_loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
          test_loss, acc, len(test_loader.dataset),
          100. * acc / len(test_loader.dataset)))
    return acc / len(test_loader.dataset)


In [9]:
best_acc = 0.0 
PATH = 'model_on_roberta_wwm_oneepoch.pth'  # 定义模型保存路径  
for epoch in range(1, NUM_EPOCHS+1):  # 3个epoch
    train(model, DEVICE, train_loader, optimizer, epoch)
    acc = test(model, DEVICE, test_loader)
    if best_acc < acc: 
        best_acc = acc 
        torch.save(model.state_dict(), PATH)  # 保存最优模型
    print("acc is: {:.4f}, best acc is {:.4f}\n".format(acc, best_acc))  

Train Epoch: 1 [2400/99752 (2.38%)]	Loss: 2.483881
Train Epoch: 1 [4800/99752 (4.79%)]	Loss: 2.504553
Train Epoch: 1 [7200/99752 (7.19%)]	Loss: 2.070401
Train Epoch: 1 [9600/99752 (9.60%)]	Loss: 1.875722
Train Epoch: 1 [12000/99752 (12.00%)]	Loss: 2.135104
Train Epoch: 1 [14400/99752 (14.41%)]	Loss: 1.975475
Train Epoch: 1 [16800/99752 (16.82%)]	Loss: 2.274744
Train Epoch: 1 [19200/99752 (19.22%)]	Loss: 2.036564
Train Epoch: 1 [21600/99752 (21.63%)]	Loss: 1.787022
Train Epoch: 1 [24000/99752 (24.03%)]	Loss: 2.312529
Train Epoch: 1 [26400/99752 (26.44%)]	Loss: 1.769793
Train Epoch: 1 [28800/99752 (28.84%)]	Loss: 1.859949
Train Epoch: 1 [31200/99752 (31.25%)]	Loss: 1.656064
Train Epoch: 1 [33600/99752 (33.65%)]	Loss: 1.819979
Train Epoch: 1 [36000/99752 (36.06%)]	Loss: 2.023920
Train Epoch: 1 [38400/99752 (38.47%)]	Loss: 1.807276
Train Epoch: 1 [40800/99752 (40.87%)]	Loss: 1.732528
Train Epoch: 1 [43200/99752 (43.28%)]	Loss: 1.933527
Train Epoch: 1 [45600/99752 (45.68%)]	Loss: 1.951922
T

In [10]:

model.load_state_dict(torch.load("model_on_roberta_wwm_oneepoch.pth"))
acc = test(model, DEVICE, test_loader)





Test set: Average loss: 0.4636, Accuracy: 20623/24939 (82.69%)
